# AAA - kurzushétvége - Lineáris modell rész

In [ ]:
#adatok betöltése

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
import glob

In [ ]:
files = glob.glob('../2_data_manipulation/data/*.csv')

In [ ]:
li = []


for filename in files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)


autok_osszes_df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
#A statikus df kialakítása

autok_statikus_df = autok_osszes_df.drop(['price','price-parsed','datum'], axis=1).drop_duplicates()

###Kész!

In [ ]:
def idokinyero(df):
    #df['datum'] = pd.to_datetime(df['datum'])
    df_s = df.sort_values(by = ['datum'])
    return pd.Series({'eslo_nap': df_s['datum'].min(), 'utolso_nap': df_s['datum'].max(),
                      'eltelt_napok': (df_s['datum'].max()-df_s['datum'].min()).days,
                      'kezdeti_ar': df_s['price-parsed'].iloc[0], 'vegso_ar': df_s['price-parsed'].iloc[-1]})

In [ ]:
auto_idoben_valtozo_df = autok_osszes_df[['link','datum','price-parsed']]

In [ ]:
auto_idoben_valtozo_df['datum'] = pd.to_datetime(auto_idoben_valtozo_df['datum'])

In [ ]:
%%time
auto_idoinfo_df = auto_idoben_valtozo_df.groupby('link').apply(idokinyero)

In [ ]:
auto_idoinfo_df['arkulonbseg'] = auto_idoinfo_df['vegso_ar'] - auto_idoinfo_df['kezdeti_ar']

In [ ]:
#auto_idoinfo_df['arkulonbseg'].value_counts()
#auto_idoinfo_df.head(10)
#gb_dic['eltelt_napok'].iloc[-1]

In [ ]:
auto_idoinfo_df['sold'] = np.where(auto_idoinfo_df['utolso_nap'] < auto_idoinfo_df['utolso_nap'].max(), 1,0 )

auto_idoinfo_df['eladasig_nap'] = np.where(auto_idoinfo_df['sold'] == 1,auto_idoinfo_df['eltelt_napok'], np.nan )

auto_idoinfo_df['piacon_nap'] = np.where(auto_idoinfo_df['sold'] == 0, auto_idoinfo_df['eltelt_napok'], np.nan )



In [ ]:
auto_adat_kesz = autok_statikus_df.merge(auto_idoinfo_df.reset_index(), on='link')

In [ ]:
auto_adat_kesz.head(10)

In [ ]:
auto_adat_kesz.to_csv('data/auto_data_model.csv')